In [1]:
import numpy as np
import cv2
import torch 
import torchvision 
from engine import train_one_epoch, evaluate
from data_reading import WindowDataset
import utils
from torch.utils.data import DataLoader

In [2]:
def get_model():
    # Initialize Model, from pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    num_classes = 2 # window + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features # Number of input features for the classifier
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model.double()

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on device {device}')
model = get_model()

running on device cpu


In [4]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr = 0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [5]:
train = DataLoader(WindowDataset(), batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
print(next(iter(train))[0])
print(next(iter(train))[1])

(tensor([[[252., 248., 216.,  ..., 227., 226., 223.],
         [252., 249., 192.,  ..., 216., 217., 224.],
         [254., 253., 168.,  ..., 222., 218., 219.],
         ...,
         [ 46.,  45.,  44.,  ..., 251., 248., 245.],
         [ 39.,  38.,  36.,  ..., 248., 245., 242.],
         [ 27.,  27.,  25.,  ..., 247., 243., 240.]],

        [[253., 250., 218.,  ..., 254., 255., 255.],
         [253., 251., 194.,  ..., 243., 248., 255.],
         [255., 255., 170.,  ..., 249., 247., 250.],
         ...,
         [ 68.,  67.,  68.,  ..., 252., 249., 246.],
         [ 60.,  59.,  59.,  ..., 250., 247., 244.],
         [ 48.,  48.,  48.,  ..., 249., 245., 242.]],

        [[255., 249., 217.,  ..., 249., 251., 249.],
         [255., 250., 191.,  ..., 238., 242., 250.],
         [255., 254., 167.,  ..., 244., 242., 244.],
         ...,
         [ 55.,  54.,  55.,  ..., 244., 241., 238.],
         [ 51.,  50.,  49.,  ..., 239., 236., 233.],
         [ 41.,  41.,  38.,  ..., 238., 234., 231.]]

In [6]:
print(next(iter(train))[1][0]['labels'].dtype)

torch.int64


In [7]:
imgs, labs = next(iter(train))
model.eval()
model(imgs)

/Users/michaelyhuang/ProjectData/python/virtual-environments/ml-venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[{'boxes': tensor([[  0.0000,  58.0702, 128.0000,  96.0000],
          [  0.0000,   7.0045, 128.0000,  12.3310],
          [  0.0000,  29.3921, 128.0000,  33.0981],
          [  0.0000,  21.2894, 128.0000,  31.2540],
          [  0.0000,  18.4755, 128.0000,  20.5184],
          [  0.0000,  10.5798, 128.0000,  13.3711],
          [  0.0000,  10.6417, 128.0000,  27.8138],
          [  0.0000,  15.8499, 128.0000,  20.8540],
          [  0.0000,  13.8282, 128.0000,  15.2906],
          [  0.0000,  13.3781, 128.0000,  14.4623],
          [  0.0000,  17.3171, 128.0000,  18.8182],
          [  0.0000,  18.1124, 128.0000,  46.8857],
          [  0.0000,  26.5195, 128.0000,  39.3971],
          [ 54.7462,  85.8553, 128.0000,  96.0000],
          [  0.0000,   0.0000, 128.0000,  96.0000],
          [  0.0000,   3.2311, 128.0000,  11.1861],
          [  0.0000,  60.2874, 128.0000,  71.9090],
          [  0.0000,  53.8701, 128.0000,  78.5278],
          [  0.0000,  65.8259, 128.0000,  74.9924],
   

In [8]:
num_epochs = 40

for epoch in range(num_epochs):
    # Train for one epoch, while printing every 10 iterations
    train_one_epoch(model, optimizer, train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, train, device)

print("DONE")

({'boxes': tensor([[ 77.,  48., 127.,  95.]]), 'labels': tensor([1])},)
Epoch: [0]  [   0/1029]  eta: 11:35:23  lr: 0.000010  loss: 122.3244 (122.3244)  loss_classifier: 0.9701 (0.9701)  loss_box_reg: 0.4877 (0.4877)  loss_objectness: 120.6482 (120.6482)  loss_rpn_box_reg: 0.2184 (0.2184)  time: 40.5473  data: 0.0026
({'boxes': tensor([[ 64.,  48., 127.,  95.]]), 'labels': tensor([1])},)
